In [ ]:
from glob import glob
import pandas as pd
from os import path
import seaborn as sns

In [ ]:
pd.read_csv("benchmark_exp/eval/uni/SRCNN.csv").file.str.removesuffix(".csv")

In [ ]:
dfs = []
for f in glob("benchmark_exp/eval/uni/*.csv"):
    print(f)
    new_df = pd.read_csv(f)
    new_df['dataset'] = new_df.file.str.removesuffix(".csv")
    algorithm = path.basename(f).split(".")[0]
    new_df['algorithm'] = algorithm
    dfs.append(new_df)

In [ ]:
all_results = pd.concat(dfs, ignore_index=True).set_index(["dataset", "algorithm"])

In [ ]:
pd.concat(dfs, ignore_index=True).groupby("algorithm")['dataset'].value_counts().unstack("algorithm").dropna().shape

In [ ]:
all_results.unstack("algorithm")

In [ ]:
all_results.shape

In [ ]:
# drop extra rows that are in SRCNN
all_results = all_results.unstack("algorithm").dropna(subset=[("file", 'Sub_KNN')]).stack("algorithm", future_stack=True)

In [ ]:
pwd

In [ ]:
files = all_results.file.unique()

def load_series(files):
    results = {}
    for f in files:
        if not f.endswith(".csv"):
            # extra files from Joel
            continue
        df = pd.read_csv("benchmark_exp/TSB-AD/TSB-AD-U/" + f)
        results[f.split(".")[0]] = df
    return results

res = load_series(files)

In [ ]:
periodicity = pd.read_csv("series_length_debugging_iqr.csv")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import MultipleLocator
import numpy as np


def plot_detection(signal, label, scores=None, train=None, ax=None, linewidth=1, window_length=None):
    if train is not None:
        if signal.index.min() < train.index.max():
            signal.index = signal.index + train.index.max()
    scores = pd.Series(scores, index=signal.index)
    label = pd.Series(np.array(label), index=signal.index)
    if ax is None:
        plt.figure(figsize=(40, 5), dpi=300)
        signal_ax = plt.gca()
    else:
        signal_ax = ax
    signal_ax.set_ylabel("signal")
    a, = signal_ax.plot(signal, label='signal', c='k', linewidth=linewidth)
    b = None
    if train is not None:
        signal_ax.plot(train, label='train', c='grey', linewidth=linewidth)
    if scores is not None:
        scores_ax = plt.twinx(signal_ax)
        b, = scores_ax.plot(scores, label='scores', c='b', alpha=0.5, linewidth=linewidth)
        scores_ax.set_ylabel("scores")
    ylims = signal_ax.get_ylim()
    yrange = ylims[1] - ylims[0]
    ymin = ylims[0] - 0.1 * yrange
    ymax = ylims[1] + 0.1 * yrange
    signal_ax.set_ylim(ymin, ymax)
    for start, end in get_anomaly_regions(label):
        width = end - start
        thin_thresh = len(label) / 1e3
        width = np.maximum(width, thin_thresh)
        signal_ax.add_patch(patches.Rectangle((start, ylims[0]), width, ylims[1] - ylims[0], facecolor='red', alpha=0.4))
    red_patch = patches.Patch(color='red', label='anomaly', alpha=0.3)
    plt.legend(handles=[a, b, red_patch] if b is not None else [a, red_patch])
    if window_length is not None:
        locator = MultipleLocator(window_length)
        locator.MAXTICKS = 2000
        signal_ax.xaxis.set_minor_locator(locator)
    return signal_ax

def get_anomaly_regions(labels):
    anomaly_starts = np.where(np.diff(labels) == 1)[0] + 1
    anomaly_ends, = np.where(np.diff(labels) == -1)
    if len(anomaly_ends):
        if not len(anomaly_starts) or anomaly_ends[0] < anomaly_starts[0]:
            # we started with an anomaly, so the start of the first anomaly is the start of the lables
            anomaly_starts = np.concatenate([[0], anomaly_starts])
    if len(anomaly_starts):
        if not len(anomaly_ends) or anomaly_ends[-1] < anomaly_starts[-1]:
            # we ended on an anomaly, so the end of the last anomaly is the end of the labels
            anomaly_ends = np.concatenate([anomaly_ends, [len(labels) - 1]])
    return list(zip(anomaly_starts, anomaly_ends))

    

In [ ]:
plot_detection(res['001_NAB_data_Traffic_4_624_2087']['Data'], res['001_NAB_data_Traffic_4_624_2087']['Label'])

In [ ]:
all_results['AUC-PR'].unstack("algorithm").plot(kind="box", rot=90)

In [ ]:
auc_pr_ranks = all_results['AUC-PR'].unstack("algorithm").rank(axis=1, ascending=False)
auc_pr_ranks

In [ ]:
rank_order = auc_pr_ranks.median().sort_values().index
sns.boxplot(auc_pr_ranks, orient='h', order=rank_order)

In [ ]:
from TSB_AD.cd_plot.cd_plot_code import cd_evaluation
import matplotlib.pyplot as plt

plt.figure(figsize=(16,  4* 2), dpi=300)
_ = cd_evaluation(all_results['AUC-PR'].unstack("algorithm"), maximize_metric=True, ax=plt.gca())
plt.title("AUC-PR")

In [ ]:
seasonal_decompose = pd.read_csv("results_seasonal_decompose.csv", index_col=0)
seasonal_decompose['algorithm'] = "seasonal decompose"
seasonal_decompose = seasonal_decompose.rename(columns={'name': 'dataset'})

In [ ]:
seasonal_decompose

In [ ]:
seasonal_decompose

In [ ]:
all_results_sd = pd.concat([all_results, seasonal_decompose.set_index(["dataset", "algorithm"])])

In [ ]:
sd_subset = all_results_sd['AUC-PR'].unstack("algorithm").dropna(subset="seasonal decompose")

In [ ]:
plt.figure(figsize=(16,  4* 2), dpi=300)
_ = cd_evaluation(sd_subset, maximize_metric=True, ax=plt.gca())
plt.title("AUC-PR")

In [ ]:
sd_subset_pretty = sd_subset.T.drop(index=["Random2", "PCA", "KNN", "LOF", "Donut", "FITS", "HBOS", "SRCNN"]).rename(index=lambda x: x.replace("Sub_", "Window "))
sd_subset_pretty

In [ ]:
comparison = sd_subset_pretty.T[['SR', 'seasonal decompose', 'Decompose']]

In [ ]:
(comparison['SR'] - comparison['seasonal decompose']).sort_values()

In [ ]:
(comparison['Decompose'] - comparison['seasonal decompose']).sort_values()

In [ ]:
periodicity

In [ ]:
subset_merged = sd_subset_pretty.T.merge(periodicity[['name', 'mine_sig', 'score']], left_on="dataset", right_on="name")
subset_merged['score'] = subset_merged['score'].fillna(0)
subset_merged['mine_sig'] = subset_merged['mine_sig'].astype(float)
subset_merged = subset_merged.set_index("name").drop(columns="mine_sig")

In [ ]:
subset_merged

In [ ]:
cm = sns.clustermap(subset_merged.T, figsize=(15, 20), standard_scale=1, cbar_pos=(0.98, 0.85, 0.01, 0.1), method="ward", dendrogram_ratio=0.1)
cm.ax_heatmap.set_xticklabels(());

In [ ]:
cm = sns.clustermap(sd_subset_pretty, figsize=(15, 20), standard_scale=1, cbar_pos=(0.98, 0.85, 0.01, 0.1), method="ward", dendrogram_ratio=0.1)
cm.ax_heatmap.set_xticklabels(());

In [ ]:
plt.figure(figsize=(16,  4* 2), dpi=300)
_ = cd_evaluation(all_results['PA-F1'].unstack("algorithm"), maximize_metric=True, ax=plt.gca())
plt.title("Adjusted F1")

In [ ]:
ranks = all_results.drop(columns=["file", "Time"]).unstack("dataset").rank(ascending=False).stack("dataset").unstack("algorithm").mean().unstack().T

In [ ]:

pd.set_option("display.precision", 1)
ranks.sort_values("AUC-PR")

In [ ]:
plt.figure(figsize=(16,  4* 2), dpi=300)
_ = cd_evaluation(all_results['AUC-ROC'].unstack("algorithm"), maximize_metric=True, ax=plt.gca())
plt.title("AUC-ROC")

In [ ]:
plt.figure(figsize=(16,  4* 2), dpi=300)
_ = cd_evaluation(all_results['VUS-ROC'].unstack("algorithm"), maximize_metric=True, ax=plt.gca())
plt.title("VUS-ROC")

In [ ]:
plt.figure(figsize=(16,  4* 2), dpi=300)
_ = cd_evaluation(all_results['VUS-PR'].unstack("algorithm"), maximize_metric=True, ax=plt.gca())
plt.title("VUS-PR")

In [ ]:
import numpy as np

In [ ]:
plt.figure(figsize=(16,  4* 2), dpi=300)
_ = cd_evaluation(all_results['PA-F1'].unstack("algorithm"), maximize_metric=True, ax=plt.gca())
plt.title("PA-F1")

In [ ]:
from benchmark_exp.HP_list import Optimal_Uni_algo_HP_dict

In [ ]:
algs = all_results.index.levels[1]

In [ ]:
[x for x in Optimal_Uni_algo_HP_dict.keys() if x not in algs]

In [ ]:
vus_pr_ranks = all_results['VUS-PR'].unstack("algorithm").rank(axis=1, ascending=False)
vus_rank_order = vus_pr_ranks.median().sort_values().index
sns.boxplot(vus_pr_ranks, orient='h', order=vus_rank_order)

In [ ]:
#sns.clustermap(all_results['VUS-PR'].unstack("algorithm"), figsize=(20, 20))

In [ ]:
# sns.clustermap(all_results['VUS-PR'].unstack("algorithm").rank(axis=1, ascending=False), figsize=(15, 20))

In [ ]:
auc_pr = all_results['AUC-ROC'].unstack("algorithm").T
cm = sns.clustermap(auc_pr, figsize=(15, 20), standard_scale=1, cbar_pos=(0.9, 0.8, 0.01, 0.1))
cm.ax_heatmap.set_xticklabels(());

In [ ]:
vus_pr = all_results['VUS-PR'].unstack("algorithm").T
cm = sns.clustermap(vus_pr, figsize=(15, 20), standard_scale=1, cbar_pos=(0.9, 0.8, 0.01, 0.1))
cm.ax_heatmap.set_xticklabels(());

In [ ]:
# auc_pr = all_results['PA-F1'].unstack("algorithm").T
# cm = sns.clustermap(auc_pr, figsize=(15, 20), standard_scale=1, cbar_pos=(0.9, 0.8, 0.01, 0.1))
# cm.ax_heatmap.set_xticklabels(());

In [ ]:
auc_pr = all_results['AUC-PR'].unstack("algorithm").T
auc_pr_sub = auc_pr.drop(index=["Random2", "PCA", "KNN", "LOF", "Donut", "FITS", "HBOS", "SRCNN"]).rename(index=lambda x: x.replace("Sub_", "Window "))

In [ ]:
auc_pr_sub

In [ ]:
cm = sns.clustermap(auc_pr_sub, figsize=(15, 20), cbar_pos=(0.98, 0.85, 0.01, 0.1), method="ward", dendrogram_ratio=0.1)
cm.ax_heatmap.set_xticklabels(());

In [ ]:
for col in ['season_iqr', 'trend_iqr', 'res_iqr']:
    periodicity.loc[np.array(~np.isfinite(periodicity[col])), col] = 0

In [ ]:
periodicity[['season_iqr', 'trend_iqr', 'res_iqr']].max(axis=1).median()

In [ ]:
periodicity['weird'] = (periodicity[['season_iqr', 'trend_iqr', 'res_iqr']].sum(axis=1) > 1.5).astype(float)

In [ ]:
periodicity

In [ ]:
#auc_pr_sub_score = auc_pr_sub.T.merge(periodicity[['name', 'r2', 'season_iqr', 'trend_iqr', 'res_iqr', 'weird']].fillna(0), left_on="dataset", right_on="name").set_index("name").T
auc_pr_sub_score = auc_pr_sub.T.merge(periodicity[['name',  'weird']].fillna(0), left_on="dataset", right_on="name").set_index("name").T

In [ ]:
cm = sns.clustermap(auc_pr_sub_score, figsize=(15, 20), cbar_pos=(0.98, 0.85, 0.01, 0.1), method="ward", dendrogram_ratio=0.1, vmax=1)
cm.ax_heatmap.set_xticklabels(());

In [ ]:
threshold = auc_pr_sub.quantile(0.8)

In [ ]:
(auc_pr_sub > threshold).T.mean().sort_values()

In [ ]:
good = (auc_pr_sub > threshold).T[['Window PCA', 'SR']].T.max()
good.mean()

In [ ]:
(auc_pr_sub > threshold).loc[:, ~good].T.mean().sort_values()

In [ ]:
good2 = (auc_pr_sub > threshold).T[['Window PCA', 'SR', 'Window KMeansAD']].T.max()
good2.mean()

In [ ]:
(auc_pr_sub > threshold).loc[:, ~good2].T.mean().sort_values()

In [ ]:
(auc_pr_sub > threshold).T[['Window PCA', 'SR', 'Window KMeansAD', 'Window LOF']].T.max().mean()

In [ ]:
bla = (auc_pr_sub > threshold).T[['Window KMeansAD', 'Window PCA', 'SR']].T.max()

In [ ]:
(auc_pr_sub > threshold).loc[:, ~bla].T.mean().sort_values()

In [ ]:
from scipy.cluster.hierarchy import fcluster
import numpy as np

In [ ]:
clusters = fcluster(cm.dendrogram_col.calculated_linkage, criterion='maxclust', t=20)

In [ ]:
datasets= [x.split("_")[1] for x in auc_pr.columns]
adjusted_rand_score(datasets, clusters)

In [ ]:
from TSB_AD.utils.slidingWindows import find_length_rank

In [ ]:
auc_pr = all_results['AUC-PR'].unstack("algorithm").T
cm = sns.clustermap(auc_pr_sub_score, figsize=(15, 20), cbar_pos=(0.9, 0.85, 0.01, 0.1), col_colors=plt.cm.tab20(clusters / clusters.max()), method="ward")
cm.ax_heatmap.set_xticklabels(());

In [ ]:
pd.Series(clusters).value_counts()

In [ ]:
clusters == 6

In [ ]:
def get_score(method, dataset):
    return np.load(f"benchmark_exp/score/uni/{method}/{d}.npy")

In [ ]:
from TSB_AD.utils.slidingWindows import find_length_rank
for d in auc_pr.columns[clusters == 3]:
    score_SR = get_score("Sub_KNN", d)
    window_length = find_length_rank(res[d]['Data'])
    plot_detection(res[d]['Data'], res[d]['Label'], scores=score_SR, window_length=window_length)
    plt.title(f"Sub_KNN: {d} ({window_length})")
    score_SR = get_score("Random2", d)
    plot_detection(res[d]['Data'], res[d]['Label'], scores=score_SR, window_length=window_length)
    plt.title(f"Sub_IForest: {d} ({window_length})")

In [ ]:
from sklearn.cluster import SpectralCoclustering, SpectralBiclustering


In [ ]:
from numpy.linalg import svd

In [ ]:
sco = SpectralBiclustering(n_clusters=(5, 8))
auc_pr_scaled = (auc_pr - auc_pr.min()) / (auc_pr.max() - auc_pr.min())
sco.fit(auc_pr_scaled)

In [ ]:
u, s, vt = svd(auc_pr_scaled)

In [ ]:
u.shape

In [ ]:
vt.shape

In [ ]:
from sklearn.decomposition import PCA
X_pca = PCA().fit_transform(auc_pr_scaled)

In [ ]:
for i, a in enumerate(auc_pr.index):
    plt.text(X_pca[i,0],  X_pca[i, 1], a)
plt.scatter(X_pca[:,0], X_pca[:, 1])

In [ ]:
X_pca = PCA().fit_transform(auc_pr_scaled.T)
datasets= [x.split("_")[1] for x in auc_pr.columns]
plt.scatter(X_pca[:,0], X_pca[:, 1], c=pd.Series(datasets, dtype="category").cat.codes, cmap=plt.cm.tab20)

In [ ]:
from umap import UMAP

In [ ]:
datasets_umap = UMAP(spread=2).fit_transform(auc_pr_scaled.T)

In [ ]:
plt.scatter(datasets_umap[:,0], datasets_umap[:, 1], c=pd.Series(datasets, dtype="category").cat.codes, cmap=plt.cm.tab20)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
datasets_tsne = TSNE(perplexity=100).fit_transform(auc_pr_scaled.T)

In [ ]:
tsne_df = pd.concat([pd.DataFrame(datasets_tsne), pd.Series(datasets, name="dataset")], axis=1)

In [ ]:
tsne_df

In [ ]:
fig = sns.scatterplot(data=tsne_df, x=0, y=1, hue="dataset", ax=plt.gca(), palette=plt.cm.tab20(range(18)))
sns.move_legend(fig, (1, -.5))

In [ ]:
sco.row_labels_[row_inds]

In [ ]:
row_inds = np.argsort(sco.row_labels_)

In [ ]:
col_inds = np.argsort(sco.column_labels_)

In [ ]:
plt.figure(figsize=(10, 10), dpi=300)
plt.imshow(np.array(auc_pr_scaled)[row_inds][:, col_inds], aspect="auto")
plt.yticks(range(len(auc_pr_scaled)), auc_pr.index[row_inds]);

In [ ]:
np.array(auc_pr_scaled).shape

In [ ]:
u,s,vt = svd(auc_pr_scaled)
row_inds = np.argsort(u[:, 2])
col_inds = np.argsort(vt[:, 0])
plt.figure(figsize=(10, 10), dpi=300)
plt.imshow(np.array(auc_pr_scaled)[row_inds][:, col_inds], aspect="auto")
plt.yticks(range(len(auc_pr_scaled)), auc_pr.index[row_inds]);

In [ ]:
cm = sns.clustermap(auc_pr, figsize=(15, 20), standard_scale=1, cbar_pos=(0.9, 0.8, 0.01, 0.1), row_clusters=sco.row_labels_, col_cluster=sco.column_labels_)
cm.ax_heatmap.set_xticklabels(());

In [ ]:
from sklearn.metrics.cluster import contingency_matrix, adjusted_rand_score

In [ ]:
datasets= [x.split("_")[1] for x in auc_pr.columns]
adjusted_rand_score(datasets, clusters)

In [ ]:
datasets= [x.split("_")[1] for x in auc_pr.columns]
plt.imshow(contingency_matrix(datasets, clusters))
plt.yticks(range(len(np.unique(datasets))), np.unique(datasets));

In [ ]:
np.bincount(clusters)

In [ ]:
for d in auc_pr.columns[clusters == 1]:
    plot_detection(res[d]['Data'], res[d]['Label'])
    plt.title(d)

In [ ]:
!ls

In [ ]:
for d in auc_pr.columns[clusters == 2]:
    scores = np.load(f"benchmark_exp/score/uni/MatrixProfile/{d}.npy")
    plot_detection(res[d]['Data'], res[d]['Label'], scores)
    plt.title(d)

In [ ]:
for d in auc_pr.columns[clusters == 3]:
    plot_detection(res[d]['Data'], res[d]['Label'])
    plt.title(d)

In [ ]:
for d in auc_pr.columns[clusters == 4]:
    plot_detection(res[d]['Data'], res[d]['Label'])
    plt.title(d)

In [ ]:
for d in auc_pr.columns[clusters == 5]:
    plot_detection(res[d]['Data'], res[d]['Label'])
    plt.title(d)

In [ ]:
def plot_subset(datasets):
    pd.Series([x.split("_")[1] for x in datasets]).value_counts().plot(kind="bar")

In [ ]:
plot_subset(auc_pr.columns[clusters == 1])

In [ ]:
plot_subset(auc_pr.columns[clusters == 2])

In [ ]:
plot_subset(auc_pr.columns[clusters == 3])

In [ ]:
plot_subset(auc_pr.columns[clusters == 4])

In [ ]:
plot_subset(auc_pr.columns[clusters == 5])

In [ ]:
bla = all_results['AUC-PR'].unstack("algorithm").rank(axis=1, ascending=False).mean().sort_values().reset_index()

In [ ]:
bla

In [ ]:
sns.clustermap(all_results['AUC-PR'].unstack("algorithm").rank(axis=1, ascending=False).T, figsize=(15, 20))

In [ ]:
vus_scores = all_results['VUS-PR'].unstack("algorithm")

In [ ]:
vus_scores.T.apply(pd.Series.idxmax)

In [ ]:
keep_datasets = vus_scores.T.apply(pd.Series.idxmax).isin(["Sub_PCA", "POLY"])

In [ ]:
_ = cd_evaluation(vus_scores[keep_datasets].drop(columns=["Sub_PCA", "POLY"]), maximize_metric=True)

In [ ]:
all_results['VUS-PR'].unstack("algorithm")

In [ ]:
sns.clustermap(all_results['VUS-PR'].unstack("algorithm")[['POLY', "Sub_PCA", "Sub_IForest"]].T.rank(), figsize=(20, 20))

In [ ]:
sns.clustermap(all_results['VUS-PR'].unstack("algorithm")[['POLY', "Sub_PCA", "Sub_IForest"]].T, figsize=(20, 20))